In [ ]:
!pip3 install --upgrade pip
!pip install wordfreq
!pip install nltk
!pip install transformers datasets torch accelerate
!pip install pyarrow

In [ ]:
import os
import re
import glob
import random
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.ipc as ipc
import pyarrow.dataset as ds
from datasets import load_dataset
from datasets import load_from_disk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading in the datasets! 😍


In [ ]:
# Change based on personal file path
dataset_folder = '/content/drive/MyDrive/CDS/paper-moderation-SP25'
# dataset_folder = 'content/drive/MyDrive/CDS/paper-moderation-SP25'

train_dataset = load_from_disk(dataset_folder + "/new_train.arrow")
val_dataset = load_from_disk(dataset_folder + "/new_val.arrow")
test_dataset = load_from_disk(dataset_folder + "/new_test.arrow")

Sanity Check!! 😤


In [ ]:
train_df = train_dataset.to_pandas()
train_df.head()
val_df = val_dataset.to_pandas()
val_df.head()
test_df = test_dataset.to_pandas()
test_df.head()

,text,labels
0,1 Protecting people in aerospace automotive te...,0
1,Atomic-scale Structural Chemical Characterizat...,1
2,Polymorphic System HD 87646 Its Evolving Stars...,0
3,Surprises in numerical expressions of physical...,1
4,Meshless RBF pseudospectral solution of acoust...,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import Dataset
from transformers import T5Tokenizer, T5Model, TrainingArguments, Trainer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

This is our preprocess function! 💕 💞

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
def preprocess_function(examples):
    # Create input prompts using the paper text (no evidence mention)
    inputs = [
        (
            "You are a helpful assistant that classifies research papers.\n\n"
            "Given the abstract and body of a research paper, predict whether the paper is from arXiv (label 1) or viXra (label 0).\n\n"
            "Respond in the following format:\n"
            "Label: <0 or 1>\n\n"
            "Paper:\n" + text
        )
        for text in examples["text"]
    ]

    # Create hard targets (1 for arXiv, 0 for viXra)
    targets = [
        f"Label: {label}"
        for label in examples["labels"]
    ]

    # Tokenize the inputs (the prompt)
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # Tokenize the targets
    with tokenizer.as_target_tokenizer():
        tokenized_labels = tokenizer(
            targets,
            max_length=8,  # very short now ("Label: 1")
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = tokenized_labels["input_ids"]
    return {
        "input_ids": model_inputs["input_ids"],
        "attention_mask": model_inputs["attention_mask"],
        "labels": tokenized_labels["input_ids"]
    }

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# Tokenize datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/27499 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/9166 [00:00<?, ? examples/s]

Making sure its on GPU! 😎

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Testing if we are using a GPU
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))
!nvidia-smi

CUDA available: True
Device: cuda
Tue Apr 29 23:56:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
!pip install --upgrade transformers

In [ ]:
import transformers
print(transformers.__version__)

4.51.3


These are our training arguments! 😇

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    predict_with_generate=True,
)

This is our compute metrics function!! 🖤

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode token IDs → strings
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=False)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Extract just the "Label: ..." integer from each prediction
    def extract_label(text):
        match = re.search(r"Label:\s*([01])", text)
        if match:
            return int(match.group(1))
        return 0  # fallback if pattern not found

    pred_ints = [extract_label(pred) for pred in decoded_preds]
    label_ints = [extract_label(label.strip()) for label in decoded_labels]

    return {
        "accuracy": accuracy_score(label_ints, pred_ints),
        "precision": precision_score(label_ints, pred_ints, zero_division=0),
        "recall": recall_score(label_ints, pred_ints, zero_division=0),
    }

Training time! ⏳ ⌛

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# ---- Step 11: Train the model ----
trainer.train()

<ipython-input-52-0eebedcf0c52>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,0.012700,0.012525,0.975671,0.967030,0.979813


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=6875, training_loss=0.009401723584261808, metrics={'train_runtime': 1762.4839, 'train_samples_per_second': 15.602, 'train_steps_per_second': 3.901, 'total_flos': 5111802988855296.0, 'train_loss': 0.009401723584261808, 'epoch': 1.0})

In [ ]:
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9166 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
predictions = trainer.predict(test_dataset)

Example Predictions! 😄

In [91]:
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)

# Get indices for some random examples
indices = random.sample(range(len(decoded_preds)), 10)
for i in indices:
    print(f"Example {i+1}:")
    print("Prediction:")
    print(decoded_preds[i])
    print("Label:")
    print(decoded_labels[i])
    print("-" * 40)

Example 5515:
Prediction:
Label: 0
Label:
0
----------------------------------------
Example 1675:
Prediction:
Label: 1
Label:
1
----------------------------------------
Example 1520:
Prediction:
Label: 1
Label:
1
----------------------------------------
Example 6225:
Prediction:
Label: 1
Label:
1
----------------------------------------
Example 1585:
Prediction:
Label: 1
Label:
1
----------------------------------------
Example 5882:
Prediction:
Label: 0
Label:
0
----------------------------------------
Example 5636:
Prediction:
Label: 0
Label:
0
----------------------------------------
Example 4334:
Prediction:
Label: 0
Label:
0
----------------------------------------
Example 712:
Prediction:
Label: 1
Label:
1
----------------------------------------
Example 7528:
Prediction:
Label: 1
Label:
1
----------------------------------------


In [92]:
def compute_metrics_test(decoded_preds,decoded_labels):

    # Extract just the "Label: ..." integer from each prediction
    def extract_label(text):
        if "0" in text:
          return 0
        return 1

    pred_ints = [extract_label(pred) for pred in decoded_preds]
    label_ints = [extract_label(label.strip()) for label in decoded_labels]

    return {
        "accuracy": accuracy_score(label_ints, pred_ints),
        "precision": precision_score(label_ints, pred_ints, zero_division=0),
        "recall": recall_score(label_ints, pred_ints, zero_division=0),
    }

Predictions! 💡

In [88]:
compute_metrics_test(decoded_preds,decoded_labels)

{'accuracy': 0.9654156665939341,
 'precision': 0.9428571428571428,
 'recall': 0.9834174477289113}